EN ESTE SISTEMA SE ENCONTRO OPERACIONES

| Información requerida                   | Información obtenida                              |
|-----------------------------------------|--------------------------------------------------|
| negocio                                 |             INFORMACION NO ENCONTRADA                                    |
| nombre cliente o razón social           |            NOMBRE                                |
| número cliente                          |              CLAVE                             |
| estatus                                 | WHERE NOMBRE NOT LIKE '%INACTI%' AND NOMBRE NOT LIKE '%APLICA%'  |    
| número contrato                         |                      FOLIO DE OPERACION                |
| género                                  |                    N/A                              |
| fecha_nacimiento o de constitución      |                     DESCONOCIDO                             |
| entidad federativa nacimiento           |                 CVE_CIUDAD                                 |
| país de nacimiento                      |          CVE_PAIS                                        |
| nacionalidad                            |              SE MAPEA CON PAIS              |
| profesión                               |                  INFORMACION NO ENCONTRADA                               |
| calle                                   |                 DIRECCION                |
| número_exterior                         |                    DIRECCION                                  |
| número_interior                         |                        DIRECCION                              |
| colonia_urbanización                    |                      DIRECCION                                |
| delegación_municipio                    |                     DIRECCION                                 |
| ciudad_población                        |                         CIUDAD                         |
| entidad_federativa                      |    INFORMACION NO ENCONTRADA                               |
| código postal                           |INFORMACION NO ENCONTRADA|
| país                                    |       INFORMACION NO ENCONTRADA                         |
| teléfono                                |    TELEFONO                                |
| correo_electrónico                      |              FAX                                    |
| rfc                                     |   INFORMACION NO ENCONTRADA  |
| curp                                    |              INFORMACION NO ENCONTRADA                                    |
| fiel                                    |                         INFORMACION NO ENCONTRADA                         |
| representante legal                     |     INFORMACION NO ENCONTRADA                                             |
| tipo de persona                         |            PM/SON BANCOS                         |
| producto contratado                     |         A DEFINIR POR OPERACIONES          |


In [3]:
from utils.tf import *
from utils.db import *
from utils.gd import * 
from utils.clt_repetidos import *
from datetime import datetime
import cx_Oracle
import pandas as pd
import os
from rapidfuzz import process, fuzz
import numpy as np


In [12]:
df = pd.read_csv("/home/ale1726/proyects/datalake/clientes/data/Clientes_activos_SIMS.dat")

In [13]:
df["RFC"] = df["RFC"].apply(lambda x: x if pd.notna(x) else "XXXXXXXXXXXXX")

In [5]:
df.columns

Index(['NEGOCIO', 'NOMBRE_O_RAZON_SOCIAL', 'NUMERO_CLIENTE', 'ESTATUS',
       'NUMERO_CONTRATO', 'GENERO', 'FECHA_NAC_O_CONST',
       'ENTIDAD_FEDERATIVA_NACIMIENTO', 'PAIS_DE_NACIMIENTO', 'NACIONALIDAD',
       'PROFESION', 'CALLE', 'NUMERO_EXTERIOR', 'NUMERO_INTERIOR',
       'COLONIA_URBANIZACION', 'DELEGACION_MUNICIPIO', 'CIUDAD_POBLACION',
       'CVE_CIUDAD', 'ENTIDAD_FEDERATIVA', 'CODIGO_POSTAL', 'PAIS', 'TELEFONO',
       'CORREO_ELECTRONICO', 'RFC', 'CURP', 'FIEL', 'REPRESENTANTE_LEGAL',
       'TIPO_PERSONA', 'PRODUCTO_CONTRATADO', 'SISTEMA_ORIGEN'],
      dtype='object')

In [14]:
df["CORREO_ELECTRONICO"] = df["CORREO_ELECTRONICO"].astype(str).apply(lambda x: x if "@" in x else pd.NA)

In [15]:
df["CIUDAD_POBLACION"] = df.apply(lambda row: "NEW YORK , N.Y." if row["CVE_CIUDAD"] == "N.Y" else row["CIUDAD_POBLACION"], axis= 1)

In [16]:
df = df.drop(columns=['CVE_CIUDAD'])

In [17]:
pais_a_nacionalidad = {
    "ESTADOS UNIDOS": "ESTADOUNIDENSE",
    "MEXICO": "MEXICANO",
    "GRAN BRETA&A": "BRITÁNICO",
    "FRANCIA": "FRANCÉS",
    "HONG KONG": "HONGKONÉS",
    "LUXEMBURGO": "LUXEMBURGUÉS"
}

In [18]:
df["NACIONALIDAD"] =  df["PAIS"].apply(lambda x: pais_a_nacionalidad[x] if x in pais_a_nacionalidad.keys() else pd.NA)

In [19]:
df.value_counts('PAIS')

PAIS
ESTADOS UNIDOS    59
MEXICO            17
GRAN BRETA&A       7
FRANCIA            1
HONG KONG          1
LUXEMBURGO         1
Name: count, dtype: int64

In [23]:
df["TELEFONO"] = df["TELEFONO"].apply(lambda x: x.replace("-", "") if pd.notna(x) else x)

In [8]:
print(df["NOMBRE_O_RAZON_SOCIAL"].value_counts().to_string(max_rows=None))

NOMBRE_O_RAZON_SOCIAL
BANCO NACIONAL DE MEXICO, S.A.                   2
INDEVAL, NAFIN                                   2
NAFIN/CEDEL                                      2
BANCO BILVAO VISCAYA                             1
NAFIN, LONDON                                    1
SCOTIA BANK                                      1
FIRST UNION BANK                                 1
BANCO NACIONAL DE COMERCIO EXTERIOR,SNC          1
BANCOMER GEOGETOWN                               1
AJG INVESTMENTS HONG KONG LTD (MERRILL LYNCH)    1
SOCIETE GENERALE PARIS  - SFI -                  1
CBI ( PERSHING)                                  1
BANK OF BOSTON, BOSTON                           1
PRUDENCIAL SEC.                                  1
BANCO MEXICANO SA GEORGETOWN                     1
DOMINION SECURITIES, TORONTO                     1
BANCO INBURSA                                    1
FIRST CAPITAL SECURITIES                         1
EURO BROKERS                                     1
ENLACE IN

In [14]:
NOMBRE_NO_VALIDOS = ["NO APLICA", "N/A"]

In [24]:
clt_repetidos = compara_nombres(df, df, "NOMBRE_O_RAZON_SOCIAL", "NOMBRE_O_RAZON_SOCIAL","NUMERO_CLIENTE","NUMERO_CLIENTE", 95)
df_resul, drops_ids = agregar_asociaciones_clientes(clt_repetidos,df)

/home/ale1726/proyects/datalake/clientes/clientes_eda/utils/clt_repetidos.py:182: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_resul = df_resul.replace("-", np.nan)


In [25]:
clt_repetidos

,ID1,NUM_CLIENTE_1,CLIENTE_1,ID2,NUM_CLIENTE_2,CLIENTE_2
22,21,DEUTSC,DEUTSCHE BANK,50,DEUT,DEUTCHE BANK
48,46,BANMEX,"BANCO NACIONAL DE MEXICO, S.A.",57,BANAME,"BANCO NACIONAL DE MEXICO, S.A."
51,48,8330,"INDEVAL, NAFIN",49,8326,"INDEVAL, NAFIN"
61,55,29653,NAFIN/CEDEL,62,37575,NAFIN/CEDEL


In [26]:
df_resul

,NEGOCIO,NOMBRE_O_RAZON_SOCIAL,NUMERO_CLIENTE,ESTATUS,NUMERO_CONTRATO,GENERO,FECHA_NAC_O_CONST,ENTIDAD_FEDERATIVA_NACIMIENTO,PAIS_DE_NACIMIENTO,NACIONALIDAD,...,PAIS,TELEFONO,CORREO_ELECTRONICO,RFC,CURP,FIEL,REPRESENTANTE_LEGAL,TIPO_PERSONA,PRODUCTO_CONTRATADO,SISTEMA_ORIGEN
0,NaN,DEUTSCHE BANK,"DEUTSC, DEUT",VIGENTE,NaN,NaN,NaN,NaN,NaN,"MEXICANO, ESTADOUNIDENSE",...,"MEXICO, ESTADOS UNIDOS",52018148,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,NaN,SIMS
1,NaN,"BANCO NACIONAL DE MEXICO, S.A.","BANMEX, BANAME",VIGENTE,NaN,NaN,NaN,NaN,NaN,MEXICANO,...,MEXICO,"12263664, 12263177","sduarte@banamex.com, yannick.polaco@citi.com",XXXXXXXXXXXXX,NaN,NaN,NaN,PM,NaN,SIMS
2,NaN,"INDEVAL, NAFIN","8330, 8326",VIGENTE,NaN,NaN,NaN,NaN,NaN,MEXICANO,...,MEXICO,NaN,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,NaN,SIMS
3,NaN,NAFIN/CEDEL,"29653, 37575",VIGENTE,NaN,NaN,NaN,NaN,NaN,MEXICANO,...,MEXICO,NaN,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,NaN,SIMS


In [27]:
clt_repetidos = compara_nombres(df, df, "NOMBRE_O_RAZON_SOCIAL", "NOMBRE_O_RAZON_SOCIAL","NUMERO_CLIENTE","NUMERO_CLIENTE", 95)
df_resul, drops_ids = agregar_asociaciones_clientes(clt_repetidos,df)
df = df.drop(drops_ids)
df = pd.concat([df, df_resul], ignore_index=True)
df = df.sort_values("NOMBRE_O_RAZON_SOCIAL")

/home/ale1726/proyects/datalake/clientes/clientes_eda/utils/clt_repetidos.py:182: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_resul = df_resul.replace("-", np.nan)


In [28]:
name_archivo = os.path.join("/home/ale1726/proyects/datalake/clientes/data/ETL","CLientesT_SIMS.csv")
df.to_csv(name_archivo,index=False)